In [6]:
import pandas as pd
URL_CSV = "../data/GVFC_extension_multimodal.csv"
df_urls = pd.read_csv(URL_CSV, usecols=["id", "article_url", "headline"])

# list of dicts
URLS = df_urls.to_dict("records")


In [7]:
import aiohttp, asyncio, async_timeout, random, os, aiofiles, trafilatura
from tqdm.asyncio import tqdm_asyncio
from newspaper import Article as NPArticle
import requests    

UA = "Mozilla/5.0 (ResearchProject/2.0 +https://your‑email)"
HEADERS = {"User-Agent": UA}
SAVE_HTML = "../data/raw_html/{id}.html"
SAVE_TXT  = "../data/full_text/{id}.txt"
LOG_CSV   = "../data/scrape_log.csv"

In [8]:
async def fetch_html(session, url, timeout=15):      # ←★ FIXED (no async_timeout)
    try:
        async with session.get(url, headers=HEADERS, timeout=timeout) as r:
            if r.status != 200:
                return None, r.status
            return await r.text(), r.status
    except Exception as e:
        print(f"Error fetching {url}: {e}")
        return None, str(e)

def make_amp_variants(url):                          # ←★ NEW
    url = url.rstrip("/")
    yield url + "/amp"
    yield url + "/amp.html"
    if "://www." in url:
        yield url.replace("://www.", "://m.")
        yield url.replace("://www.", "://amp.")

def good(txt, min_words=100):                        # ←★ NEW
    return isinstance(txt, str) and len(txt.split()) >= min_words

def try_extract(html, url):
    """Run trafilatura then newspaper3k; return text or None."""
    txt = trafilatura.extract(html, url=url, include_comments=False)
    if good(txt):
        return txt
    art = NPArticle(url)
    art.set_html(html)
    try:
        art.parse()
        return art.text if good(art.text) else None
    except Exception:
        return None

In [9]:
async def scrape_article(session, rec):
    art_id, url = rec["id"], rec["article_url"]
    txt_path = SAVE_TXT.format(id=art_id)
    if os.path.exists(txt_path):           # already scraped
        return "cached"

    # 0) ORIGINAL URL --------------------------------------------------------
    html, status = await fetch_html(session, url)
    # print(f"[{art_id}] original status = {status}, url = {url}")
    if html and status == 200:
        text = try_extract(html, url)
        if good(text):
            # cache html (optional)
            async with aiofiles.open(SAVE_HTML.format(id=art_id), "w") as f:
                await f.write(html)
            async with aiofiles.open(txt_path, "w") as f:
                await f.write(text)
            return "ok"

    # 1) AMP / mobile variants ---------------------------------------------  ←★ NEW
    for alt in make_amp_variants(url):
        html2, status2 = await fetch_html(session, alt)
        # print(f"[{art_id}] AMP status = {status2}, url = {alt}")
        if html2 and status2 == 200:
            text2 = try_extract(html2, alt)
            if good(text2):
                async with aiofiles.open(txt_path, "w") as f:
                    await f.write(text2)
                return "ok_amp"

    # 2) 12ft.io proxy ------------------------------------------------------  ←★ NEW
    proxy_url = f"https://12ft.io/proxy?q={url}"
    html3, status3 = await fetch_html(session, proxy_url)
    # print(f"[{art_id}] 12ft status = {status3}, url = {proxy_url}")
    if html3 and status3 == 200:
        text3 = try_extract(html3, url)        # keep original url for metadata
        if good(text3):
            async with aiofiles.open(txt_path, "w") as f:
                await f.write(text3)
            return "ok_12ft"

    # if everything failed ---------------------------------------------------
    return ("fail_extract", status)

async def scrape_all(url_records):
    async with aiohttp.ClientSession() as session:
        results = []
        for rec in tqdm_asyncio(url_records, total=len(url_records)):
            res = await scrape_article(session, rec)
            results.append((rec["id"], rec["article_url"], res))
            # politeness: 0.5‑1.5 s gap
            await asyncio.sleep(random.uniform(0.5, 1.5))
        return results

# ---------------------------------------------------------------------------

scrape_results = await scrape_all(URLS)              # (inside Jupyter cell)
pd.DataFrame(scrape_results,
             columns=["id", "url", "status"]
            ).to_csv(LOG_CSV, index=False)


  1%|          | 11/1300 [00:25<41:41,  1.94s/it] 

Error fetching https://amp.nytimes.com/2018/02/16/us/politics/fact-check-parkland-gun-violence-mental-illness.html?partner=rss&emc=rss: Cannot connect to host amp.nytimes.com:443 ssl:default [nodename nor servname provided, or not known]


  3%|▎         | 36/1300 [01:22<46:13,  2.19s/it]  

Error fetching https://amp.nytimes.com/2018/08/04/us/parkland-florida-nikolas-cruz.html: Cannot connect to host amp.nytimes.com:443 ssl:default [nodename nor servname provided, or not known]


  5%|▍         | 59/1300 [02:05<54:19,  2.63s/it]

...

 98%|█████████▊| 1278/1300 [1:20:16<01:06,  3.00s/it]

Error fetching https://m.chicagotribune.com/news/local/breaking/ct-chicago-hospital-shooting-nra-20181121-story.html: 
Error fetching https://amp.chicagotribune.com/news/local/breaking/ct-chicago-hospital-shooting-nra-20181121-story.html: Cannot connect to host amp.chicagotribune.com:443 ssl:default [nodename nor servname provided, or not known]


 98%|█████████▊| 1280/1300 [1:20:38<02:09,  6.46s/it]

Error fetching https://m.chicagotribune.com/suburbs/oak-park/news/ct-oak-park-gun-storage-tl-1115-story.html: 
Error fetching https://amp.chicagotribune.com/suburbs/oak-park/news/ct-oak-park-gun-storage-tl-1115-story.html: Cannot connect to host amp.chicagotribune.com:443 ssl:default [nodename nor servname provided, or not known]


100%|██████████| 1300/1300 [1:21:51<00:00,  3.78s/it]
